In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
import fitz
import os
import boto3
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

F:\anaconda3\envs\python_3_7\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



# Code to Convert PDF to PNG

In [2]:
def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/New/'
        output_file = f"{path+os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [3]:
convert_pdf2img('Makro Invoices/Makro Invoices/Server_Invoice/document_98_document_22119_174-documentos.pdf')

Deprecation: 'preRotate' removed from class 'Matrix' after v1.19 - use 'prerotate'.
Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


## Summary ########################################################
File:Makro Invoices/Makro Invoices/Server_Invoice/document_98_document_22119_174-documentos.pdf
Pages:None
Output File(s):['Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page1.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page2.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page3.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page4.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page5.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page6.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page7.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page8.png', 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page9.png', 'Makro I

['Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page1.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page2.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page3.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page4.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page5.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page6.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page7.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page8.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page9.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page10.png',
 'Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page11.png',
 'Makro 

In [3]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [6]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [4]:
Ima = "Makro Invoices/Makro Invoices/New/document_98_document_22119_174-documentos_page7.png"

In [5]:
def find_amounts(text):
    amounts = re.findall(r'\d+\.\d{2}\b', text)
    floats = [float(amount) for amount in amounts]
    unique = list(dict.fromkeys(floats))
    if len(unique)==0:
        return 0
    else:
        return max(unique)

In [6]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
       "Document_Type":"",
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
tot = []
doc_type = []
print(type(text_list))
dat['Total']=find_amounts(text.replace(',','.'))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        

        a = a.replace(',','.')
        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
            print(b)
            
        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
            print(b)
            
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('-',''))

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0])
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b[0].replace('/',''))
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            di = b[0].replace('-','')
            dit = di.replace(',','')
            dat["Vat Number"] = dit.replace('.','')
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            di = b[0].replace('-','')
            dit = di.replace(',','')
            dat["Vat Number"] = dit.replace('/','')

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
        
        if "ALBARAN" in a or "Albaran" in a:
            dat["Document_Type"] = "Albaran"
            doc_type.append("Albaran")
        elif "Factura" in a or "FACTURA" in a  or "factura" in a :
            dat["Document_Type"] = "Factura"
            doc_type.append("Factura")
        if len(doc_type)>1:
            dat["Document_Type"] = doc_type[0]
        
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        elif (re.search("\d{1}/\d{2}/\d{2}",a)):
            b = re.findall("\d{1}/\d{2}/\d{2}",a)
            dat["Date"] = b[0].replace('/','-')
            date.append(b[0].replace('/','-'))
        
        elif (re.search("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)
            dat["Date"] = b[0].replace('.','-')
            date.append(b[0].replace('.','-'))
    
        if len(date)>1:
            dat["Date"] = date[0].replace('/','-')

    
    
       
        
        

<class 'list'>
['B57538183']
['B62558568']
Wall time: 2.13 s


In [7]:
dat

{'Vat Number': 'B57538183',
 'Total': 173.8,
 'Date': '02-06-2022',
 'Document_Type': '',
 'Invoice_Number': ''}

In [8]:
print(dat['Date'][0:6])

02-06-


In [9]:
dat
if len(dat['Date'])==10:
    pass
elif len(dat['Date'])==8:
    dat['Date'] = dat['Date'][0:6]+'20'+dat['Date'][6:]

In [10]:
dat

{'Vat Number': 'B57538183',
 'Total': 173.8,
 'Date': '02-06-2022',
 'Document_Type': '',
 'Invoice_Number': ''}

In [11]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number  Total        Date Document_Type Invoice_Number
0  B57538183  173.8  02-06-2022                             


In [12]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data['Vat Number']= data['Vat Number'].str.replace('/', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until



,Vat Number,Total,Date,Document_Type,Invoice_Number
0,B57538183,173.8,02-06-2022,,


In [304]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

NameError: name 'formats' is not defined

In [14]:
df2 = formats.query("last == True")

In [15]:
df2

,formato,nombre,nif,last
8892,\d\d\d\d\d\d\d,Isayca Tenerife SLU,B38988242,True
12519,\d\d\d\d\d\d\d,Isayca Tenerife SLU,B38988242,True


In [16]:
inv = 0
for i in df2['formato']:
    inv = i
if inv ==0:
    pass
else:
    factura = re.compile(inv)
    mo = factura.search(text)
    if mo is None:
        pass
    else:
        dat["Invoice_Number"]=mo.group()

In [17]:
dat

{'Vat Number': 'B38988242',
 'Total': '273.22',
 'Date': '3-02-20',
 'Document_Type': 'Factura',
 'Invoice_Number': '1825768'}

In [13]:
im = Image.open(Ima)

In [14]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

ClientError: An error occurred (UnrecognizedClientException) when calling the AnalyzeDocument operation: The security token included in the request is invalid.

In [ ]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [1538]:
len(dataframes)

2

In [1539]:
d = dataframes[0]

In [1540]:
d

,Código,Descripción,Lote,Fecha,Ig,Bultos,Unid.,Precio,Dto,Importe
0,99078,SALEROS MESA BOTE 200gr.,27705,,"3,0",If,"25,000","0,480",,"12,00"
1,20015,ACEITE ALTO OLEICO 25L. 60% th,12116110,01/02/21,"0,0",3,"3,000","28,750",,"86,25"
2,20003,"ABRILSEM GRRFA. 25L. ,",4431209,01/09/19,"0,0",1',"1,000","23,750",,"23,75"
3,20460,MAYONESA SALSA RICA 10Kg.,190325,19/03/25,"3,0",2',"2,000","10,280",,"20,56"
4,99281,MERMELADA ARANDANOS 5KG.,181119,18/11/19,"3,0",1,"1,000","23,900",,"23,90"
5,25200,VINO BRIK BLANCO 1L.,A030DA,,"7,0",1',"12,000","0,790",,"9,48"
6,30005,ZANAHORIA RALLADA 5KG,160924,16/09/24,"3,0",2,"6,000","2,120",,"12,72"
7,35460,TOMATE FRITO APIS 3KG,2281916,16/08/23,"3,0",1',"6,000","1,650",,"9,90"
8,35456,T.TRITURADO APIS 5KG.,2181916,06/08/23,"3,0",1',"3,000","2,280",,"6,84"
9,20525,SALSA DE SOJA 1L.,191108,08/11/22,"3,0",,"1,000","2,900",,"2,90"


In [1541]:
d.columns

Index(['Código', 'Descripción', 'Lote', 'Fecha', 'Ig', 'Bultos', 'Unid.',
       'Precio', 'Dto', 'Importe'],
      dtype='object')

In [1542]:
merged_col = False
print(type(d))
for i in (d.columns):
    if i == '':
        print(i)

        merged_col = True

<class 'pandas.core.frame.DataFrame'>


In [1543]:
a= d[d.iloc[:,1].str.contains("Lot")==True]

In [1544]:
a

,Código,Descripción,Lote,Fecha,Ig,Bultos,Unid.,Precio,Dto,Importe


In [1545]:
for i in d.iloc[:,0]:
    if 'Lote' in i:
        print('yes')

In [1546]:
if "Código Descripción" in d.columns:
    print('HI')

In [1547]:
def get_desc(desc):
    return desc.split(" ")[0].strip(" ")

In [1548]:
if merged_col:
    print("hi")
    if "Código Descripción" in d.columns:
        d['Codigo'] = d["Código Descripción"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Código Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
    else:
        d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
        d['Codigo']= d['Codigo'].str.replace('/', '')
        d['Codigo']= d['Codigo'].str.replace(':', '')
        d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
        d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
        d['DESCRIPCIÓN']= d['Descripción'].str.replace('[0-9]', '')
        d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+' '+d.iloc[:,1]
        d =d.drop(d.columns[[0, 1]],axis = 1) 
        first_column = d.pop('Codigo')
        d.insert(0,'Codigo',first_column)
        second_column = d.pop('DESCRIPCIÓN')
        d.insert(1,'DESCRIPCIÓN',second_column)
        
        
#    d['Codigo'] = d["Código"].apply(lambda x: f"{get_desc(x)}")
#    d['Codigo']= d['Codigo'].str.replace('/', '')
#    d['Codigo']= d['Codigo'].str.replace('[A-Z]', '')
#    d['Codigo']= d['Codigo'].str.replace('[a-z]', '')
#    d['DESCRIPCIÓN']= d['Código'].str.replace('/', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN'].str.replace('[0-9]', '')
#    d['DESCRIPCIÓN']= d['DESCRIPCIÓN']+d.iloc[:,1]
#    d =d.drop(d.columns[[0, 1]],axis = 1) 
#    first_column = d.pop('Codigo')
#    d.insert(0,'Codigo',first_column)
#    second_column = d.pop('DESCRIPCIÓN')
#    d.insert(1,'DESCRIPCIÓN',second_column)

In [1549]:
if merged_col:
    this_data = d
    print('Yes')

In [1550]:
str1 ='DESCRIPCIÓN'
str2 = "Descrip. artículo"
str3 = "Artículo/ Cod.cliente"
str4 = 'DESCRIPCIÓN:'
str5 = "CONCEPTO"
str6 = "UDS"
str7 = "Concepto"
str8 = "Descripción"
str9 = "DESCRIPCION"   
str10 = "Description articulo"
str11 = "Description"
str12 = "DESCRIPCIÓN ARTICULO"
str13 = "Unidades Descripción"
str14 = "PRODUCTO"
str15 = "Articulo"
str16 = 'Denominación /Referencia'
str17 = 'DESCRIPCIÓN'
str18 = 'REF.'
str19 = "Codigo"
str20 = "Lote"
str21 = "MM Num. articulo"
str22 = 'Artículo'
zero = False
one = False
two = False
three = False

if not merged_col or len(dataframes)>0:
    if "ARTÍCULO" in dataframes[0].columns or "CODI EAN/CÓDIGO EAN" in dataframes[0].columns or "MM Num. artículo" in dataframes[0].columns  or 'CÓDIGO' in dataframes[0].columns or 'MM Num.' in dataframes[0].columns or str3 in dataframes[0].columns or str4 in dataframes[0].columns or str5 in dataframes[0].columns or str6 in dataframes[0].columns or str7 in dataframes[0].columns or str8 in dataframes[0].columns or str9 in dataframes[0].columns or str10 in dataframes[0].columns or str11 in dataframes[0].columns or str12 in dataframes[0].columns or str13 in dataframes[0].columns or str14 in dataframes[0].columns or str15 in dataframes[0].columns or str16 in dataframes[0].columns or str17 in dataframes[0].columns or str18 in dataframes[0].columns or str19 in dataframes[0].columns or str20 in dataframes[0].columns or str21 in dataframes[0].columns or str22 in dataframes[0].columns:
            this_data = pd.DataFrame(dataframes[0])
            print('---------0')
            zero = True
    if len(dataframes)>1: 
        if "ARTÍCULO" in dataframes[1].columns or "CODI EAN/CÓDIGO EAN" in dataframes[1].columns or "MM Num. artículo" in dataframes[1].columns or 'CÓDIGO' in dataframes[1].columns or 'MM Num.' in dataframes[1].columns or str3 in dataframes[1].columns or str4 in dataframes[1].columns or str5 in dataframes[1].columns or str6 in dataframes[1].columns or str7 in dataframes[1].columns or str8 in dataframes[1].columns or str9 in dataframes[1].columns or str10 in dataframes[1].columns or str11 in dataframes[1].columns or str12 in dataframes[1].columns or str13 in dataframes[1].columns or str14 in dataframes[1].columns or str15 in dataframes[1].columns or str16 in dataframes[1].columns or str17 in dataframes[1].columns or str18 in dataframes[1].columns or str19 in dataframes[1].columns or str20 in dataframes[1].columns or str21 in dataframes[1].columns or str22 in dataframes[1].columns:        
                this_data = pd.DataFrame(dataframes[1])
                print('---------1')
                one = True
    if len(dataframes)>2:        
        if "ARTÍCULO" in dataframes[2].columns or "CODI EAN/CÓDIGO EAN" in dataframes[2].columns or "MM Num. artículo" in dataframes[2].columns or 'CÓDIGO' in dataframes[2].columns or 'MM Num.' in dataframes[2].columns or str3 in dataframes[2].columns or str4 in dataframes[2].columns or str5 in dataframes[0].columns or str6 in dataframes[2].columns or str7 in dataframes[2].columns or str8 in dataframes[2].columns or str9 in dataframes[2].columns or str10 in dataframes[2].columns or str11 in dataframes[2].columns or str12 in dataframes[2].columns or str13 in dataframes[2].columns or str14 in dataframes[2].columns or str15 in dataframes[2].columns or str16 in dataframes[2].columns or str16 in dataframes[2].columns or str18 in dataframes[2].columns or str19 in dataframes[2].columns or str20 in dataframes[2].columns or str21 in dataframes[2].columns or str22 in dataframes[2].columns:
                this_data = pd.DataFrame(dataframes[2])
                print('---------2')
                two = True
    if len(dataframes)>3:
        if "ARTÍCULO" in dataframes[3].columns or "CODI EAN/CÓDIGO EAN" in dataframes[3].columns or "MM Num. artículo" in dataframes[3].columns or 'CÓDIGO' in dataframes[3].columns or 'MM Num.' in dataframes[3].columns or str3 in dataframes[3].columns or str4 in dataframes[3].columns or str5 in dataframes[3].columns or str6 in dataframes[3].columns or str7 in dataframes[3].columns or str8 in dataframes[3].columns or str9 in dataframes[3].columns or str10 in dataframes[3].columns or str11 in dataframes[3].columns or str12 in dataframes[3].columns or str13 in dataframes[3].columns or str14 in dataframes[3].columns or str15 in dataframes[3].columns or str16 in dataframes[3].columns or str16 in dataframes[3].columns or str18 in dataframes[3].columns or str19 in dataframes[3].columns or str20 in dataframes[3].columns or  str21 in dataframes[3].columns or str22 in dataframes[3].columns:
            this_data = pd.DataFrame(dataframes[3])
            print('---------3')
            three = True
else:
    pass
    
this_data


---------0


,Código,Descripción,Lote,Fecha,Ig,Bultos,Unid.,Precio,Dto,Importe
0,99078,SALEROS MESA BOTE 200gr.,27705,,"3,0",If,"25,000","0,480",,"12,00"
1,20015,ACEITE ALTO OLEICO 25L. 60% th,12116110,01/02/21,"0,0",3,"3,000","28,750",,"86,25"
2,20003,"ABRILSEM GRRFA. 25L. ,",4431209,01/09/19,"0,0",1',"1,000","23,750",,"23,75"
3,20460,MAYONESA SALSA RICA 10Kg.,190325,19/03/25,"3,0",2',"2,000","10,280",,"20,56"
4,99281,MERMELADA ARANDANOS 5KG.,181119,18/11/19,"3,0",1,"1,000","23,900",,"23,90"
5,25200,VINO BRIK BLANCO 1L.,A030DA,,"7,0",1',"12,000","0,790",,"9,48"
6,30005,ZANAHORIA RALLADA 5KG,160924,16/09/24,"3,0",2,"6,000","2,120",,"12,72"
7,35460,TOMATE FRITO APIS 3KG,2281916,16/08/23,"3,0",1',"6,000","1,650",,"9,90"
8,35456,T.TRITURADO APIS 5KG.,2181916,06/08/23,"3,0",1',"3,000","2,280",,"6,84"
9,20525,SALSA DE SOJA 1L.,191108,08/11/22,"3,0",,"1,000","2,900",,"2,90"


In [1551]:
print(zero)

True


In [1552]:
below_table = ''
index = 0
if len(dataframes)>1:
    if zero:
        below_table = dataframes[1]
        index = 1
        print('---------')
if len(dataframes)>2:    
    if one:
        below_table = dataframes[2]
        index = 2
        print('---------')
        
if len(dataframes)>3:
    if two:
        below_table = dataframes[3]
        index = 3
        print('---------')
if len(dataframes)>3:
    if three:
        below_table = dataframes[4]
        index = 4
        print('---------')

---------


In [1553]:
below_table
#dataframes[3]

,Subtotal,Descuento,Base Imponible,%,Igic,
0,"268,98",,"140,10","0,0",,
1,,,"119,40","3,0","3,58",
2,,,"9,48","7,0","0,66",
3,,,,,,"TOTAL 273,22"


In [1554]:
len(below_table)

4

In [1555]:
if  len(below_table)>0:
    print(below_table.columns)

Index(['Subtotal', 'Descuento', 'Base Imponible', '%', 'Igic', ''], dtype='object')


IMPORT/IMPORTE

In [1556]:
check = False
if len(below_table)>0:
    if "Base Imponible"  in below_table.columns or "% I.V.A"  in below_table.columns or "Cuota I.V.A."  in below_table.columns or "BASE IMP."  in below_table.columns or "I.V.A."  in below_table.columns or "%"  in below_table.columns or "CUOTA"  in below_table.columns or "TOTAL BRUTO"  in below_table.columns or "%IVA"  in below_table.columns or "B.IMPONIBLE"  in below_table.columns or "CUOTA IVA"  in below_table.columns or "% R.E."  in below_table.columns or "BASE"  in below_table.columns or "TOTAL IMP."  in below_table.columns:
        check = True
        print(check)
    

True


In [1575]:
check = False
if len(dataframes)>1:
    if  'Igic' in dataframes[1].columns or "% IMP" in dataframes[1].columns or "IVA" in dataframes[1].columns or "IMPORT/IMPORTE" in dataframes[1].columns or "Base Imponible"  in dataframes[1].columns or "% I.V.A"  in dataframes[1].columns or "Cuota I.V.A."  in dataframes[1].columns or "BASE IMP."  in dataframes[1].columns or "I.V.A."  in dataframes[1].columns or "%"  in dataframes[1].columns or "CUOTA"  in dataframes[1].columns or "TOTAL BRUTO"  in dataframes[1].columns or "%IVA"  in dataframes[1].columns or "B.IMPONIBLE"  in  dataframes[1].columns or "CUOTA IVA"  in  dataframes[1].columns or "% R.E."  in  dataframes[1].columns or "BASE"  in  dataframes[1].columns or "TOTAL IMP."  in  dataframes[1].columns:
        check = True
        below_table = dataframes[1]
        print(check)
        print(below_table)
if len(dataframes)>2:
    print('YEs')
    if 'Igic' in dataframes[2].columns or "% IMP" in dataframes[2].columns or "IVA" in dataframes[2].columns or "IMPORT/IMPORTE" in dataframes[2].columns or "Base Imponible"  in dataframes[2].columns or "% I.V.A"  in dataframes[2].columns or "Cuota I.V.A."  in dataframes[2].columns or "BASE IMP."  in dataframes[2].columns or "I.V.A."  in dataframes[2].columns or "%"  in dataframes[2].columns or "CUOTA"  in dataframes[2].columns or "TOTAL BRUTO"  in dataframes[2].columns or "%IVA"  in dataframes[2].columns or "B.IMPONIBLE"  in  dataframes[2].columns or "CUOTA IVA"  in  dataframes[2].columns or "% R.E."  in  dataframes[2].columns or "BASE"  in  dataframes[2].columns or "TOTAL IMP."  in  dataframes[2].columns:
        check = True
        below_table = dataframes[2]
        print(check)
        print(below_table)
elif len(dataframes)>2:    
    if 'Igic' in dataframes[3].columns or "% IMP" in dataframes[3].columns or "IVA" in dataframes[3].columns or "IMPORT/IMPORTE" in dataframes[3].columns or "Base Imponible"  in dataframes[3].columns or "% I.V.A"  in dataframes[3].columns or "Cuota I.V.A."  in dataframes[3].columns or "BASE IMP."  in dataframes[3].columns or "I.V.A."  in dataframes[3].columns or "%"  in dataframes[3].columns or "CUOTA"  in dataframes[3].columns or "TOTAL BRUTO"  in dataframes[3].columns or "%IVA"  in dataframes[3].columns or "B.IMPONIBLE"  in  dataframes[3].columns or "CUOTA IVA"  in  dataframes[3].columns or "% R.E."  in  dataframes[3].columns or "BASE"  in  dataframes[3].columns or "TOTAL IMP."  in  dataframes[3].columns:
        check = True
        below_table = dataframes[3]
        print(check)
        print(below_table)
elif len(dataframes)>3:    
    if 'Igic' in dataframes[4].columns or "% IMP" in dataframes[4].columns or "IVA" in dataframes[4].columns or "IMPORT/IMPORTE" in dataframes[4].columns or "Base Imponible"  in dataframes[4].columns or "% I.V.A"  in dataframes[4].columns or "Cuota I.V.A."  in dataframes[4].columns or "BASE IMP."  in dataframes[4].columns or "I.V.A."  in dataframes[4].columns or "%"  in dataframes[4].columns or "CUOTA"  in dataframes[4].columns or "TOTAL BRUTO"  in dataframes[4].columns or "%IVA"  in dataframes[4].columns or "B.IMPONIBLE"  in  dataframes[4].columns or "CUOTA IVA"  in  dataframes[4].columns or "% R.E."  in  dataframes[4].columns or "BASE"  in  dataframes[4].columns or "TOTAL IMP."  in  dataframes[4].columns:
        check = True
        below_table = dataframes[4]
        print(check)
        print(below_table)
        
elif len(dataframes)>4:    
    if 'Igic' in dataframes[5].columns or "IVA" in dataframes[5].columns or "IMPORT/IMPORTE" in dataframes[5].columns or "Base Imponible"  in dataframes[5].columns or "% I.V.A"  in dataframes[5].columns or "Cuota I.V.A."  in dataframes[5].columns or "BASE IMP."  in dataframes[5].columns or "I.V.A."  in dataframes[5].columns or "%"  in dataframes[5].columns or "CUOTA"  in dataframes[5].columns or "TOTAL BRUTO"  in dataframes[5].columns or "%IVA"  in dataframes[5].columns or "B.IMPONIBLE"  in  dataframes[5].columns or "CUOTA IVA"  in  dataframes[5].columns or "% R.E."  in  dataframes[5].columns or "BASE"  in  dataframes[5].columns or "TOTAL IMP."  in  dataframes[5].columns:
        check = True
        below_table = dataframes[5]
        print(check)
        print(below_table)
elif len(dataframes)>5:    
    if 'Igic' in dataframes[6].columns or "IVA" in dataframes[6].columns or "IMPORT/IMPORTE" in dataframes[6].columns or "Base Imponible"  in dataframes[6].columns or "% I.V.A"  in dataframes[6].columns or "Cuota I.V.A."  in dataframes[6].columns or "BASE IMP."  in dataframes[6].columns or "I.V.A."  in dataframes[6].columns or "%"  in dataframes[6].columns or "CUOTA"  in dataframes[6].columns or "TOTAL BRUTO"  in dataframes[6].columns or "%IVA"  in dataframes[6].columns or "B.IMPONIBLE"  in  dataframes[6].columns or "CUOTA IVA"  in  dataframes[6].columns or "% R.E."  in  dataframes[6].columns or "BASE"  in  dataframes[6].columns or "TOTAL IMP."  in  dataframes[6].columns:
        check = True
        below_table = dataframes[6]
        print(check)
        print(below_table)

True
  Subtotal Descuento Base Imponible    %  Igic              
0   268,98                   140,10  0,0                    
1                            119,40  3,0  3,58              
2                              9,48  7,0  0,66              
3                                               TOTAL 273,22


In [1574]:
#i = index
if len(dataframes)>1:
    print('hi')

hi


In [1568]:
len(dataframes)

2

In [1569]:
#if i<=len(dataframes)-1 and check==False:
#        print('T')
#        below_table = dataframes[index+2]

In [1570]:
dataframes[1].columns

Index(['Subtotal', 'Descuento', 'Base Imponible', '%', 'Igic', ''], dtype='object')

In [1571]:

#if "Base Imponible" not in below_table.columns or "% I.V.A" not in below_table.columns or "Cuota I.V.A." not in below_table.columns or "BASE IMP." not in below_table.columns or "I.V.A." not in below_table.columns or "%" not in below_table.columns or "CUOTA" not in below_table.columns or "TOTAL BRUTO" not in below_table.columns or "%IVA" not in below_table.columns or "B.IMPONIBLE" not in below_table.columns or "CUOTA IVA" not in below_table.columns or "% R.E." not in below_table.columns or "BASE" not in below_table.columns or "TOTAL IMP." not in below_table.columns:
#    if i<len(dataframes):
#        print('T')
#        below_table = dataframes[index+1]

In [1572]:
below_table

,Subtotal,Descuento,Base Imponible,%,Igic,
0,"268,98",,"140,10","0,0",,
1,,,"119,40","3,0","3,58",
2,,,"9,48","7,0","0,66",
3,,,,,,"TOTAL 273,22"


In [1181]:
if len(this_data)>4:
    try:
        this_data.columns = this_data.columns.str.replace('SELECTED  ', '')
        this_data.columns = this_data.columns.str.replace('SELECTED ', '')
        this_data.columns = this_data.columns.str.replace('SELECTED', '')
        this_data.columns = this_data.columns.str.replace('NOT_', '')
        this_data.columns = this_data.columns.str.replace('NOT_SELECTED  ', '')
        this_data.columns = this_data.columns.str.replace('NOT_SELECTED', '')
        big = this_data.columns[0]
        cont = this_data.columns[2]
        lot = this_data.columns[1]
        prec = this_data.columns[3]
        cant = this_data.columns[-5]
        imp = this_data.columns[-2]
        importe = this_data.columns[-3]
        third = this_data.columns[-4]
        this_data = this_data.loc[(this_data[big] != "Número de pedido") & (this_data[big] != "*** Número de pedido") & (this_data[big]!="*** Número de") & (this_data[big]!="*** Número")]
        this_data = this_data.loc[(this_data[big] != "Entregado a: TABERNA") & (this_data[big] !="Entregado a: TASCA") & (this_data[big]!="Entregado a: TABERNA OSUNA")
              & (this_data[big]!= "Entregado a: TABERNA osuna")& (this_data[big]!="Fin de número") ]
        this_data = this_data.loc[(this_data[imp]!= "Spain Fecha:") & (this_data[imp]!= "Spain Fecha:0") & (this_data[imp]!= "Spain Fecha:1")]
        this_data = this_data.loc[this_data[cont] !="LOT:"]
        this_data = this_data.loc[this_data[cant] !="Total"]
        this_data = this_data.loc[this_data[cant]!="LA LAGUNA" ]
        this_data = this_data.replace('',np.nan)
        this_data = this_data.dropna(axis=0, thresh=3, how="any")
        limitPer = len(this_data) * .100
        this_data = this_data.dropna(thresh=limitPer, axis=1)
        this_data = this_data.replace(np.nan,'0')
    except:
        pass

In [1182]:
data = this_data
big = this_data

In [1183]:
#for i in this_data.columns:
#    if i == '':
#        this_data.drop(i,  inplace=True)
if len(big.columns)==3:
    print('Hi')

In [1184]:
this_data.head()

,Codigo,Lote,c o N c E P T o,Cantidad UM,Precio,$IVA,Importe,Sección Al
0,44710,22-05-31,HUMUS 2 KG.,"2,000 UD","12,310",10,"24,62",CONSERVA 1
1,39176,22-10-31,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000 UD","1,810",10,"10,86",CONSERVA 1
2,28438,23-09-30,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000 UD","5,730",10,"11,46",CONSERVA I
3,28483,23-09-30,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000 UD","8,870",10,"26,61",CONSERVA I
4,40461,23-08-31,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000 UD","26,670",10,"80,01",CONSERVA I


In [1185]:
# for  second column
this_data["second_count"] = this_data.iloc[:, 1].str.count("\D")
this_data["second_length"] = this_data.iloc[:, 1].str.len()
this_data["second_digit_percentage"] = this_data["second_count"] / this_data["second_length"] * 100



In [1186]:
# for third column
this_data["third_count"] = this_data.iloc[:, 2].str.count("\D")
this_data["third_length"] = this_data.iloc[:, 2].str.len()
this_data["third_digit_percentage"] = this_data["third_count"] / this_data["third_length"] * 100



In [1187]:
len(this_data.columns)

14

In [1188]:
this_data.head()

,Codigo,Lote,c o N c E P T o,Cantidad UM,Precio,$IVA,Importe,Sección Al,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,44710,22-05-31,HUMUS 2 KG.,"2,000 UD","12,310",10,"24,62",CONSERVA 1,2,8,25.0,10,11,90.909091
1,39176,22-10-31,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000 UD","1,810",10,"10,86",CONSERVA 1,2,8,25.0,35,37,94.594595
2,28438,23-09-30,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000 UD","5,730",10,"11,46",CONSERVA I,2,8,25.0,40,40,100.000000
3,28483,23-09-30,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000 UD","8,870",10,"26,61",CONSERVA I,2,8,25.0,34,35,97.142857
4,40461,23-08-31,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000 UD","26,670",10,"80,01",CONSERVA I,2,8,25.0,37,40,92.500000


In [1189]:
# for  forth column
dftwo = False
if len(data.columns)==3:
    this_data["forth_count"] = this_data.iloc[:, 3].str.count("\D")
    this_data["forth_length"] = this_data.iloc[:, 3].str.len()
    this_data["forth_digit_percentage"] = this_data["forth_count"] / this_data["forth_length"] * 100
    print('Im True')
    dftwo = True



In [1190]:
this_data.fillna(0)

,Codigo,Lote,c o N c E P T o,Cantidad UM,Precio,$IVA,Importe,Sección Al,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,44710,22-05-31,HUMUS 2 KG.,"2,000 UD","12,310",10,"24,62",CONSERVA 1,2,8,25.0,10,11,90.909091
1,39176,22-10-31,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000 UD","1,810",10,"10,86",CONSERVA 1,2,8,25.0,35,37,94.594595
2,28438,23-09-30,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000 UD","5,730",10,"11,46",CONSERVA I,2,8,25.0,40,40,100.000000
3,28483,23-09-30,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000 UD","8,870",10,"26,61",CONSERVA I,2,8,25.0,34,35,97.142857
4,40461,23-08-31,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000 UD","26,670",10,"80,01",CONSERVA I,2,8,25.0,37,40,92.500000
5,40464,23-04-30,ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA,"6,000 UD","20,930",10,"125,58",CONSERVA I,2,8,25.0,37,40,92.500000
6,42037,29-06-30,"SAL MALDON CUBO DE 1,4 KG. MALDON","3,000 UD","20,500",10,"61,50",CONSERVA I,2,8,25.0,31,33,93.939394
7,38112,23-03-01,SALMON AHUMADO PRECORTADO NORUEGO---- KG,"2,000 KG","19,950",10,"39,90",CONGELADO 1,2,8,25.0,40,40,100.000000
8,67600,23-09-30,"McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG","37,500 KG","2,310",4,"86,63",CONGELADO 1,2,8,25.0,31,35,88.571429
9,42033,22-10-30,"FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK","5,000 UD","14,640",10,"73,20",CONGELADO I,2,8,25.0,32,40,80.000000


In [1191]:
df2 = ''
df = this_data.query("second_digit_percentage>30.0")
df1 = this_data.query("third_digit_percentage>30.0")
if dftwo:
    df2.fillna(0)
    df2 = this_data.query("forth_digit_percentage>60.0")


In [1192]:
df.shape

(0, 14)

In [1193]:
#df2 = this_data.query("second_digit_percentage>30.0")
this_data.shape

(11, 14)

In [1194]:
this_data.head()

,Codigo,Lote,c o N c E P T o,Cantidad UM,Precio,$IVA,Importe,Sección Al,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,44710,22-05-31,HUMUS 2 KG.,"2,000 UD","12,310",10,"24,62",CONSERVA 1,2,8,25.0,10,11,90.909091
1,39176,22-10-31,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000 UD","1,810",10,"10,86",CONSERVA 1,2,8,25.0,35,37,94.594595
2,28438,23-09-30,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000 UD","5,730",10,"11,46",CONSERVA I,2,8,25.0,40,40,100.000000
3,28483,23-09-30,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000 UD","8,870",10,"26,61",CONSERVA I,2,8,25.0,34,35,97.142857
4,40461,23-08-31,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000 UD","26,670",10,"80,01",CONSERVA I,2,8,25.0,37,40,92.500000


In [1195]:
r ,c= df.shape
print(r,c)

0 14


In [1196]:
r1 ,c1= df1.shape
print(r1,c1)

11 14


In [1197]:
len(df2)

0

In [1198]:
if len(df2)>0:
    r2 ,c2= df2.shape
    print(r2,c2)

In [1199]:
desc = ''
if len(df.columns) == len(this_data.columns)and r>0:
    desc = df.iloc[:,1]
    print(' I got in ')
elif len(df1.columns)== len(this_data.columns):
    desc = df1.iloc[:,2]
    print('I got it ')
elif len(df2.columns)== len(this_data.columns):
    desc = df2.iloc[:,3]
    print('I got it2 ')

I got it 


In [1200]:
desc

0                                  HUMUS 2 KG.
1        LECHE ENTERA LA CREME 1,5L. PRESIDENT
2     MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX
3          MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX
4     ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA
5     ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA
6            SAL MALDON CUBO DE 1,4 KG. MALDON
7     SALMON AHUMADO PRECORTADO NORUEGO---- KG
8          McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG
9     FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK
10    FALAFEL BANDEJA $OX35GR.1,750KG.---GREEK
Name: c o N c E P T o, dtype: object

In [1201]:
this_data.head()

,Codigo,Lote,c o N c E P T o,Cantidad UM,Precio,$IVA,Importe,Sección Al,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,44710,22-05-31,HUMUS 2 KG.,"2,000 UD","12,310",10,"24,62",CONSERVA 1,2,8,25.0,10,11,90.909091
1,39176,22-10-31,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000 UD","1,810",10,"10,86",CONSERVA 1,2,8,25.0,35,37,94.594595
2,28438,23-09-30,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000 UD","5,730",10,"11,46",CONSERVA I,2,8,25.0,40,40,100.000000
3,28483,23-09-30,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000 UD","8,870",10,"26,61",CONSERVA I,2,8,25.0,34,35,97.142857
4,40461,23-08-31,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000 UD","26,670",10,"80,01",CONSERVA I,2,8,25.0,37,40,92.500000


In [1202]:
dfff = ''
description_list = ["Descripción","Descrip. artículo","DESCRIPCION","Description articulo","Description","DESCRIPCIÓN ARTICULO","Unidades Descripción"]
if len(desc) == 0:
    print('Empty')
    for i in ((description_list)):
        if i in this_data.columns:
            print(i)
            dfff = this_data[i]    
            break
       

In [1203]:
desc

0                                  HUMUS 2 KG.
1        LECHE ENTERA LA CREME 1,5L. PRESIDENT
2     MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX
3          MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX
4     ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA
5     ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA
6            SAL MALDON CUBO DE 1,4 KG. MALDON
7     SALMON AHUMADO PRECORTADO NORUEGO---- KG
8          McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG
9     FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK
10    FALAFEL BANDEJA $OX35GR.1,750KG.---GREEK
Name: c o N c E P T o, dtype: object

In [1204]:
if len(desc)!=0:
    desc_data = pd.DataFrame(desc)
    desc_data.head()
elif len(dfff)!=0:
    desc_data = pd.DataFrame(dfff)
    desc_data.head()

In [1205]:
desc

0                                  HUMUS 2 KG.
1        LECHE ENTERA LA CREME 1,5L. PRESIDENT
2     MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX
3          MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX
4     ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA
5     ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA
6            SAL MALDON CUBO DE 1,4 KG. MALDON
7     SALMON AHUMADO PRECORTADO NORUEGO---- KG
8          McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG
9     FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK
10    FALAFEL BANDEJA $OX35GR.1,750KG.---GREEK
Name: c o N c E P T o, dtype: object

In [1206]:
split = ''

In [1207]:
desc_data.head()

,c o N c E P T o
0,HUMUS 2 KG.
1,"LECHE ENTERA LA CREME 1,5L. PRESIDENT"
2,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX
3,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX
4,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA


In [1208]:
split = desc_data
split = split.iloc[:,0].str.split("LOTE", n = 1, expand = True)

In [1209]:
split.rename(columns = {'Description':'0'}, inplace = True)

In [1210]:
split[0]

0                                  HUMUS 2 KG.
1        LECHE ENTERA LA CREME 1,5L. PRESIDENT
2     MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX
3          MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX
4     ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA
5     ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA
6            SAL MALDON CUBO DE 1,4 KG. MALDON
7     SALMON AHUMADO PRECORTADO NORUEGO---- KG
8          McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG
9     FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK
10    FALAFEL BANDEJA $OX35GR.1,750KG.---GREEK
Name: 0, dtype: object

In [1211]:
desc_data = split[0]

In [1212]:
desc_data.head()

0                                 HUMUS 2 KG.
1       LECHE ENTERA LA CREME 1,5L. PRESIDENT
2    MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX
3         MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX
4    ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA
Name: 0, dtype: object

In [1213]:
s1,s2 = split.shape
if s2>1:
    lote = split[1]
    

In [1214]:
this_data

,Codigo,Lote,c o N c E P T o,Cantidad UM,Precio,$IVA,Importe,Sección Al,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,44710,22-05-31,HUMUS 2 KG.,"2,000 UD","12,310",10,"24,62",CONSERVA 1,2,8,25.0,10,11,90.909091
1,39176,22-10-31,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000 UD","1,810",10,"10,86",CONSERVA 1,2,8,25.0,35,37,94.594595
2,28438,23-09-30,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000 UD","5,730",10,"11,46",CONSERVA I,2,8,25.0,40,40,100.000000
3,28483,23-09-30,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000 UD","8,870",10,"26,61",CONSERVA I,2,8,25.0,34,35,97.142857
4,40461,23-08-31,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000 UD","26,670",10,"80,01",CONSERVA I,2,8,25.0,37,40,92.500000
5,40464,23-04-30,ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA,"6,000 UD","20,930",10,"125,58",CONSERVA I,2,8,25.0,37,40,92.500000
6,42037,29-06-30,"SAL MALDON CUBO DE 1,4 KG. MALDON","3,000 UD","20,500",10,"61,50",CONSERVA I,2,8,25.0,31,33,93.939394
7,38112,23-03-01,SALMON AHUMADO PRECORTADO NORUEGO---- KG,"2,000 KG","19,950",10,"39,90",CONGELADO 1,2,8,25.0,40,40,100.000000
8,67600,23-09-30,"McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG","37,500 KG","2,310",4,"86,63",CONGELADO 1,2,8,25.0,31,35,88.571429
9,42033,22-10-30,"FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK","5,000 UD","14,640",10,"73,20",CONGELADO I,2,8,25.0,32,40,80.000000


In [1215]:
len(data.columns)

14

In [1216]:
this_data

,Codigo,Lote,c o N c E P T o,Cantidad UM,Precio,$IVA,Importe,Sección Al,second_count,second_length,second_digit_percentage,third_count,third_length,third_digit_percentage
0,44710,22-05-31,HUMUS 2 KG.,"2,000 UD","12,310",10,"24,62",CONSERVA 1,2,8,25.0,10,11,90.909091
1,39176,22-10-31,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000 UD","1,810",10,"10,86",CONSERVA 1,2,8,25.0,35,37,94.594595
2,28438,23-09-30,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000 UD","5,730",10,"11,46",CONSERVA I,2,8,25.0,40,40,100.000000
3,28483,23-09-30,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000 UD","8,870",10,"26,61",CONSERVA I,2,8,25.0,34,35,97.142857
4,40461,23-08-31,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000 UD","26,670",10,"80,01",CONSERVA I,2,8,25.0,37,40,92.500000
5,40464,23-04-30,ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA,"6,000 UD","20,930",10,"125,58",CONSERVA I,2,8,25.0,37,40,92.500000
6,42037,29-06-30,"SAL MALDON CUBO DE 1,4 KG. MALDON","3,000 UD","20,500",10,"61,50",CONSERVA I,2,8,25.0,31,33,93.939394
7,38112,23-03-01,SALMON AHUMADO PRECORTADO NORUEGO---- KG,"2,000 KG","19,950",10,"39,90",CONGELADO 1,2,8,25.0,40,40,100.000000
8,67600,23-09-30,"McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG","37,500 KG","2,310",4,"86,63",CONGELADO 1,2,8,25.0,31,35,88.571429
9,42033,22-10-30,"FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK","5,000 UD","14,640",10,"73,20",CONGELADO I,2,8,25.0,32,40,80.000000


In [1217]:
code  = this_data.iloc[:, 0]
#codingo = this_data[code]
codingo_data  =  pd.DataFrame(code)
codingo_data


,Codigo
0,44710
1,39176
2,28438
3,28483
4,40461
5,40464
6,42037
7,38112
8,67600
9,42033


In [1218]:
prec = []
str1 = "Precio"
str2 = "Base I"
str3 = "PVP"
str4 = 'Importe'
str5 = 'Euro Uni.'
str6 = "IMP"
price =''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        prec.append(i)
        price = i
        print("Im condition 1")
    ratio2 = fuzz.partial_ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        prec.append(i)
        price =i
        print("Im condition 2")
    ratio3 = fuzz.partial_ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        prec.append(i)
        price =i
    ratio4 = fuzz.partial_ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        prec.append(i)
        price =i
    ratio5 = fuzz.partial_ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        prec.append(i)
        price =i
    ratio6 = fuzz.partial_ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        prec.append(i)
        price =i
if len(prec)>1:
    price = prec[0]
else:
    pass

Im condition 4
Lote
Precio
Im condition 1
Im condition 4
Importe
Im condition 6
Importe


In [1219]:
if price !='':
    price_col = this_data[price]
    price_data = pd.DataFrame(price_col,columns=[price])
    price_data
else:
    print("Price Is Empty")

In [1220]:
str1 = "DTO"
str2 = "Descuento"
str3 = "DCTO"
str4 = "Desc"
disc = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        disc = i
        print("Im condition 1")
    
    elif ratio2>=75:
        print(i)
        disc = i
        print("Im condition 2")
    
    elif ratio3>=75:
        print(i)
        disc = i
        print("Im condition 3")
    
    elif ratio4>=75:
        print(i)
        disc = i
        print("Im condition 4")
    
print(disc)

In [1221]:
if disc!='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    disc_data

In [1222]:
str1 = "Cantidad"
str2 = "Cant"
str3 = "Unid"
str4 = "Unidades"
str5 = "Uds"
str6 = "Peso"
cat = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        cat = i
        print("Im condition 1")
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        cat =i
        print("Im condition 2")
    ratio3 = fuzz.ratio(str3.lower(),str(i).lower())
    if ratio3>=75:
        print("Im condition 3")
        print(i)
        cat =i
    ratio4 = fuzz.ratio(str4.lower(),str(i).lower())
    if ratio4>=75:
        print("Im condition 4")
        print(i)
        cat =i
    ratio5 = fuzz.ratio(str5.lower(),str(i).lower())
    if ratio5>=75:
        print("Im condition 5")
        print(i)
        cat =i
    ratio6 = fuzz.ratio(str6.lower(),str(i).lower())
    if ratio6>=75:
        print("Im condition 6")
        print(i)
        cat =i

Cantidad UM
Im condition 1


In [1223]:
if cat!='':
    Cantidad = this_data[cat]
    Cantidad_data = pd.DataFrame(Cantidad,columns=[cat])
    Cantidad_data
else:
    print("Quantity is Empty")
#Cantidad_data

In [1224]:
Cantidad_data

,Cantidad UM
0,"2,000 UD"
1,"6,000 UD"
2,"2,000 UD"
3,"3,000 UD"
4,"3,000 UD"
5,"6,000 UD"
6,"3,000 UD"
7,"2,000 KG"
8,"37,500 KG"
9,"5,000 UD"


In [1225]:
if cat!='':
    fecha= Cantidad_data.iloc[:,0].str.split(" ", n = 1, expand = True)
    print(Cantidad_data)

   Cantidad UM
0     2,000 UD
1     6,000 UD
2     2,000 UD
3     3,000 UD
4     3,000 UD
5     6,000 UD
6     3,000 UD
7     2,000 KG
8    37,500 KG
9     5,000 UD
10    1,000 UD


In [1226]:
Cantidad_data = fecha[0]

In [1227]:
Cantidad_data

0      2,000
1      6,000
2      2,000
3      3,000
4      3,000
5      6,000
6      3,000
7      2,000
8     37,500
9      5,000
10     1,000
Name: 0, dtype: object

In [1228]:
str1 = 'IMPUESTO'
imposto = ''
for i in this_data.columns:
    ratio1 = fuzz.partial_ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        imposto = i
print(imposto)

In [1229]:
if imposto!='':
    imposto_col = this_data[imposto]
    imposto_data = pd.DataFrame(imposto_col,columns=[imposto])
    imposto_data

In [1230]:
str1 = 'Tasa'
str2 = 'TIPO'
tasa = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        tasa = i
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        tasa = i
print(tasa)

In [1231]:
if tasa!='':
    tasa_col = this_data[tasa]
    tasa_data = pd.DataFrame(tasa_col,columns=[tasa])
    tasa_data

In [1232]:
str1 = 'Lote'
str2 = 'LOTE'
lote = ''
for i in this_data.columns:
    ratio1 = fuzz.ratio(str1.lower(),str(i).lower())
    if ratio1>=75:
        print(i)
        lote = i
    ratio2 = fuzz.ratio(str2.lower(),str(i).lower())
    if ratio2>=75:
        print(i)
        lote = i
print(lote)

Lote
Lote
Lote


In [1233]:
if lote!='':
    lote_col = this_data[lote]
    lote_data = pd.DataFrame(lote_col,columns=[lote])
    lote_data

In [1234]:
#codingo_data
#dataf
#Cantidad_data
#price_data

In [1235]:
if disc =='' and imposto =='' and tasa =='' and cat=='' and price == '':
    df2 = pd.concat([codingo_data,desc_data],axis=1, join='inner')
    df2['Price'] = '0'
    df2['Quantity'] = '0'
    df2['Discount'] = '0'
    df2['IMPUESTO']=0
    df2['Tasa']=0
   
   

elif disc!='' and imposto =='' and tasa =='':
    disc_col = this_data[disc]
    disc_data = pd.DataFrame(disc_col,columns=[disc])
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    df2['Tasa']=0
    
elif disc!='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,imposto_data],axis=1, join='inner')
    df2['Tasa']=0
    
elif disc!='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,disc_data,tasa_data],axis=1, join='inner')
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data,tasa_data],axis=1, join='inner')
    df2['Discount']='0'
    
elif disc=='' and imposto=='' and tasa!='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,tasa_data],axis=1, join='inner')
    df2['Discount']='0'
    df2['IMPUESTO']=0
    
elif disc=='' and imposto!='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data,imposto_data],axis=1, join='inner')
    df2['Discount']='0'
    df2['Tasa']=0

elif disc=='' and imposto=='' and tasa=='':
    df2 = pd.concat([codingo_data,desc_data,Cantidad_data,price_data],axis=1, join='inner')
    df2['Discount']='0'
    df2['IMPUESTO']=0
    df2['Tasa']=0

In [1236]:
df2.head()

,Codigo,0,0,Lote,Discount,IMPUESTO,Tasa
0,44710,HUMUS 2 KG.,"2,000",22-05-31,0,0,0
1,39176,"LECHE ENTERA LA CREME 1,5L. PRESIDENT","6,000",22-10-31,0,0,0
2,28438,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,"2,000",23-09-30,0,0,0
3,28483,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,"3,000",23-09-30,0,0,0
4,40461,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,"3,000",23-08-31,0,0,0


In [1237]:
df2 = df2.replace('',0)

In [1238]:
df2.columns.values[0] = "Codigo"
df2.columns.values[1] = "Description"
df2.columns.values[2] = "Cantidad"
df2.columns.values[3] = "Precio"

In [1239]:
df2.iloc[:,3]= df2.iloc[:,3].str.replace('[A-Z]', '')
df2.iloc[:,3]= df2.iloc[:,3].str.replace('[/\+%&*-]', '')
df2.iloc[:,3]= df2.iloc[:,3].str.replace(',', '.').astype(float)

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  



In [1240]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Codigo       11 non-null     object 
 1   Description  11 non-null     object 
 2   Cantidad     11 non-null     object 
 3   Precio       11 non-null     float64
 4   Discount     11 non-null     object 
 5   IMPUESTO     11 non-null     int64  
 6   Tasa         11 non-null     int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 1004.0+ bytes


In [1241]:
df2.iloc[:,2]= df2.iloc[:,2].str.replace(',', '.').astype(float)

In [1242]:
df2.iloc[:,4]= df2.iloc[:,4].str.replace(',', '.').astype(float)

In [1243]:
df2

,Codigo,Description,Cantidad,Precio,Discount,IMPUESTO,Tasa
0,44710,HUMUS 2 KG.,2.0,220531.0,0.0,0,0
1,39176,"LECHE ENTERA LA CREME 1,5L. PRESIDENT",6.0,221031.0,0.0,0,0
2,28438,MOSTAZA DIJON TRADICIONAL CUBO IKG.VILUX,2.0,230930.0,0.0,0,0
3,28483,MOSTAZA ANTIGUA PLASTICO 1 KG.VILUX,3.0,230930.0,0.0,0,0
4,40461,ACEITE OLIVA VIRGEN EX. (60)--5 L.SANDUA,3.0,230831.0,0.0,0,0
5,40464,ACEITE GIRASOL REFINADO 60)-- 5 L.SANDUA,6.0,230430.0,0.0,0,0
6,42037,"SAL MALDON CUBO DE 1,4 KG. MALDON",3.0,290630.0,0.0,0,0
7,38112,SALMON AHUMADO PRECORTADO NORUEGO---- KG,2.0,230301.0,0.0,0,0
8,67600,"McCAIN PATATAS 3/8 BOLSA 2,5 KG. KG",37.5,230930.0,0.0,0,0
9,42033,"FALAFEL BANDEJA 50X25GR.1,250KG.---GREEK",5.0,221030.0,0.0,0,0


In [69]:
df2.dtypes

Codigo          object
Description     object
Cantidad       float64
Precio         float64
Discount       float64
IMPUESTO         int64
Tasa             int64
dtype: object